In [2]:
# all necessary includes
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier

import os

In [33]:
# additional functions
def TrainClassifier(dataframe):
    X_train, Y_train = dataframe['X'], dataframe['target']
    
    model = RandomForestClassifier(
                                    # params
                                  )
    
def DepthFirstSearch(cur_folder, names, file_name, doc_text, name=''):
    # print(name, cur_folder)
    files = os.listdir()
    cur_name = name[:]
    if len(cur_name) != 0:
        cur_name += '-'
    cur_name += cur_folder
    folders = [x for x in files if '.' not in x]
    for x in folders:
        os.chdir(x)
        DepthFirstSearch(x, names, file_name, doc_text, cur_name)
        os.chdir('..')
    if len(folders) == 0:
        for file in files:
            names.append(cur_name)
            file_name.append(file)
            doc_text.append('')
        

In [34]:
# read input
# os.chdir('..')
names = []
file_name = []
doc_text = []
dataset_path = 'datasets'
os.chdir(dataset_path)
DepthFirstSearch('', names, file_name, doc_text)
os.chdir('..')
df = pd.DataFrame({'file_name': file_name, 'doc_type': names})
print(df)

  file_name doc_type
0  absd.txt  a1-f_pg


In [ ]:
# modify input


In [ ]:
# split into validation and training
validation_fraction = 0.3


In [ ]:
# train the model


In [ ]:
# write model to file


In [ ]:
# assess accuracy of the model
